In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

### Carregar dados - Regressão

In [2]:
df = pd.read_csv('./datasets/dataset_colesterol.csv')
df.drop(columns=['Id'], axis=1, inplace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Grupo Sanguíneo     996 non-null    object 
 1   Fumante             997 non-null    object 
 2   Nível de Atividade  996 non-null    object 
 3   Idade               997 non-null    float64
 4   Peso                997 non-null    float64
 5   Altura              997 non-null    float64
 6   Colesterol          1000 non-null   float64
dtypes: float64(4), object(3)
memory usage: 54.8+ KB


In [4]:
df.fillna({
  'Grupo Sanguíneo': df['Grupo Sanguíneo'].mode()[0],
  'Fumante': df['Fumante'].mode()[0],
  'Nível de Atividade': df['Nível de Atividade'].mode()[0],
  'Idade': df['Idade'].median(),
  'Peso': df['Peso'].median(),
  'Altura': df['Altura'].median()
}, inplace=True)

In [5]:
# Ajustar DataFrame
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,False,False,True,False,False,True,False,True,False
1,68.0,105.0,184.0,236.98,True,False,False,False,True,False,False,False,True
2,25.0,64.8,180.0,161.79,False,False,False,True,True,False,True,False,False
3,43.0,120.2,167.0,336.24,True,False,False,False,True,False,True,False,False
4,79.0,88.5,175.0,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,False,True,False,False,False,True,False,False,True
996,51.0,47.7,170.0,128.03,False,False,False,True,True,False,True,False,False
997,39.0,85.5,176.0,211.14,False,True,False,False,True,False,False,True,False
998,61.0,91.2,161.0,284.53,False,True,False,False,False,True,False,True,False


In [6]:
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,False,False,True,False,False,True,False,True,False
1,68.0,105.0,184.0,236.98,True,False,False,False,True,False,False,False,True
2,25.0,64.8,180.0,161.79,False,False,False,True,True,False,True,False,False
3,43.0,120.2,167.0,336.24,True,False,False,False,True,False,True,False,False
4,79.0,88.5,175.0,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,False,True,False,False,False,True,False,False,True
996,51.0,47.7,170.0,128.03,False,False,False,True,True,False,True,False,False
997,39.0,85.5,176.0,211.14,False,True,False,False,True,False,False,True,False
998,61.0,91.2,161.0,284.53,False,True,False,False,False,True,False,True,False


### Treinar modelo de Regressão linera múltipla com RFE

In [7]:
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [9]:
# Treinar modelo com RFE
# RFE (Recursive Feature Elimination)
# Uso um estimador e defino uma quantidade de features (hiperparâmetros)
# O RFE faz iterações iniciando com todas as features e eliminando a cada iteração até atingir a quantidade definida
# Elimina as características/features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [10]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Grupo Sanguíneo_B',
       'Fumante_Sim', 'Nível de Atividade_Alto'],
      dtype='object')

In [11]:
# Ranking de features
def show_ranking(method_fs, X_train):
  ranking = rfe_method.ranking_

  nomes_features = X_train.columns.to_list()

  df_ranking = pd.DataFrame({'Feature': nomes_features, 'Ranking': ranking})

  df_ranking = df_ranking.sort_values(by='Ranking')

  print(df_ranking)

In [12]:
# Ranking de features do RFE Regressão
show_ranking(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
5             Grupo Sanguíneo_B        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
7                   Fumante_Não        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [13]:
# Avaliar performance
def performance_regression(model, X_test, y_test):
  y_pred = model.predict(X_test)

  return mean_squared_error(y_test, y_pred, squared=False)

In [14]:
# Perfomance regressão com RFE
performance_regression(rfe_method, X_test, y_test)

c:\Users\Gabriel\.virtualenvs\topicos_complementares-qTRtXzs4\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.997293997492445

### Treinar modelo sem RFE

In [15]:
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [16]:
performance_regression(model_reg, X_test, y_test)

c:\Users\Gabriel\.virtualenvs\topicos_complementares-qTRtXzs4\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.040245951911174

### Treinar modelo de regressão com RFECV

In [17]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [18]:
performance_regression(rfe_method_cv, X_test, y_test)

c:\Users\Gabriel\.virtualenvs\topicos_complementares-qTRtXzs4\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.047574304437292

In [19]:
# Quais features foram escolhidas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

### Treinar modelo de regressão com SelectFromModel

In [20]:
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [21]:
# Quais features foram escolhidas
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Nível de Atividade_Alto'], dtype='object')

In [22]:
# Treinar modelo com features selecionadas
X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)

LinearRegression()

In [23]:
performance_regression(model_reg, X_test_ajustado_reg, y_test)

c:\Users\Gabriel\.virtualenvs\topicos_complementares-qTRtXzs4\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.037781310936559

### Carregar e preparar dados - Classificação

In [24]:
df2 = pd.read_csv('./datasets/fruit_quality.csv')

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


In [26]:
df2.head(10)

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good
5,5,-3.425400,-1.409082,-1.913511,-0.555775,-3.853071,1.914616,-2.981523,bad
6,6,1.331606,1.635956,0.875974,-1.677798,3.106344,-1.847417,2.414171,good
7,7,-1.995462,-0.428958,1.530644,-0.742972,0.158834,0.974438,-1.470125,good
8,8,-3.867632,-3.734514,0.986429,-1.207655,2.292873,4.080921,-4.871905,bad
9,9,-0.727983,-0.442820,-4.092223,0.597513,0.393714,1.620857,2.185608,bad


In [27]:
df2.drop('A_id', axis=1, inplace=True)

df2['Quality'] = (df2['Quality'] == 'good').astype(int)

df2

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


### Treinar modelo de Regressão Logistica com RFE

In [28]:
X = df2.drop('Quality', axis=1)
y = df2['Quality']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [54]:
# Treinar modelo com RFE
# RFE (Recursive Feature Elimination)
# Uso um estimador e defino uma quantidade de features (hiperparâmetros)
# O RFE faz iterações iniciando com todas as features e eliminando a cada iteração até atingir a quantidade definida
# Elimina as características/features menos importantes
rfe_method = RFE(estimator=LogisticRegression(), n_features_to_select=7)
rfe_method.fit(X_train, y_train)

RFE(estimator=LogisticRegression(), n_features_to_select=7)

In [39]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Ripeness', 'Acidity'], dtype='object')

In [56]:
# Ranking de features do RFE Regressão
show_ranking(rfe_method, X_train)

       Feature  Ranking
0         Size        1
1       Weight        1
2    Sweetness        1
3  Crunchiness        1
4    Juiciness        1
5     Ripeness        1
6      Acidity        1


In [42]:
# Avaliar performance
def performance_classification(model, X_test, y_test):
  y_pred = model.predict(X_test)

  return f1_score(y_test, y_pred)

In [55]:
# Perfomance regressão com RFE
performance_classification(rfe_method, X_test, y_test)

0.7787903893951947

### Treinar modelo sem RFE

In [46]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression()

In [47]:
performance_classification(model_lr, X_test, y_test)

0.7787903893951947

### Treinar modelo de regressão Logistica com RFECV

In [62]:
rfe_method_cv = RFECV(estimator=LogisticRegression(), min_features_to_select=4, cv=5, scoring='f1_weighted')
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LogisticRegression(), min_features_to_select=4,
      scoring='f1_weighted')

In [81]:
performance_classification(rfe_method_cv, X_test, y_test)

0.7767634854771784

In [59]:
# Quais features foram escolhidas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Size', 'Weight', 'Sweetness', 'Juiciness', 'Ripeness', 'Acidity'], dtype='object')

### Treinar modelo de regressão Logistica com SelectFromModel

In [78]:
sfm_method = SelectFromModel(estimator=model_lr, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LogisticRegression(), max_features=4, threshold=0.5)

In [79]:
# Quais features foram escolhidas
X_train.columns[(sfm_method.get_support())]

Index(['Size', 'Sweetness'], dtype='object')

In [74]:
# Treinar modelo com features selecionadas
X_train_ajustado_lr = sfm_method.transform(X_train)
X_test_ajustado_lr = sfm_method.transform(X_test)
model_lr.fit(X_train_ajustado_lr, y_train)

LogisticRegression()

In [80]:
performance_classification(model_lr, X_test_ajustado_lr, y_test)

0.697171381031614